##  7% of Provide region P06 is missing!!! (scenario independent) WHY???


		-

**these two glaciers fail, because of missing model_geometry_spinup_historical**

RGIId
RGI60-10.00006    13 km2
RGI60-10.00002    49 km2

- ok, the issue comes from `No such file or directory: b'/tmp/OGGM/OGGM_gcm_run/per_glacier/RGI60-10/RGI60-10.00/RGI60-10.00002/model_geometry_spinup_historical.nc'`

In [12]:
rgi_reg = '14'

# RGI glaciers 
rgi_ids = ['RGI60-10.00002', 'RGI60-10.00005', 'RGI60-10.00001', 'RGI60-10.00006', 'RGI60-10.00007', 'RGI60-10.00004', 'RGI60-10.00003', 'RGI60-10.00008']




In [13]:
import os
import logging 
import sys

# Libs
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

# Locals
import oggm.cfg as cfg
from oggm import utils, workflow, tasks
# from oggm.shop import gcm_climate
import logging
# Module logger
log = logging.getLogger(__name__)
from oggm import entity_task

#from func_add import run_hydro_from_2000_ref_area_2000_hist_w5e5_w_overshot_stab_scenarios


# normally from oggm.shop.gcm_climate import process_cmip_data
# but the changes are not yet updated ...
from oggm.shop.gcm_climate import process_gcm_data
@entity_task(log, writes=['gcm_data'])
def process_cmip_data(gdir, filesuffix='', fpath_temp=None,
                      fpath_precip=None, y0=None, y1=None,
                      **kwargs):
    """Read, process and store the CMIP5 and CMIP6 climate data for this glacier.

    It stores the data in a format that can be used by the OGGM mass balance
    model and in the glacier directory.

    Currently, this function is built for the CMIP5 and CMIP6 projection
    simulations that are on the OGGM servers.

    Parameters
    ----------
    filesuffix : str
        append a suffix to the filename (useful for ensemble experiments).
    fpath_temp : str
        path to the temp file
    fpath_precip : str
        path to the precip file
    y0 : int
        start year of the CMIP data processing.
        Default is None which processes the entire timeseries.
        Set this to the beginning of your bias correction/
        projection period minus half of bc period
        to make process_cmip_data faster.
    y1 : int
        end year of the CMIP data processing.
        Set this to the end of your projection period
        plus half of bc period. Default is None to process
        the entire time series, same as y0.
    **kwargs: any kwarg to be passed to ref:`process_gcm_data`
    """

    # Glacier location
    glon = gdir.cenlon
    glat = gdir.cenlat

    if y0 is not None:
        y0 = str(y0)
    if y1 is not None:
        y1 = str(y1)
    # Read the GCM files
    with xr.open_dataset(fpath_temp, use_cftime=True) as tempds, \
            xr.open_dataset(fpath_precip, use_cftime=True) as precipds:

        # only process and save the gcm data selected --> saves some time!
        if (y0 is not None) or (y1 is not None):
            tempds = tempds.sel(time=slice(y0, y1))
            precipds = precipds.sel(time=slice(y0, y1))
        # Check longitude conventions
        if tempds.lon.min() >= 0 and glon <= 0:
            glon += 360

        # Take the closest to the glacier
        # Should we consider GCM interpolation?
        try:
            # if gcms are not flattened, do:
            # this is the default, so try this first
            temp = tempds.tas.sel(lat=glat, lon=glon, method='nearest')
            precip = precipds.pr.sel(lat=glat, lon=glon, method='nearest')
        except:
            # are the gcms flattened? if yes,
            # compute all the distances and choose the
            # nearest gridpoint
            c_tempds = ((tempds.lon - glon) ** 2 +
                 (tempds.lat - glat) ** 2)
            c_precipds = ((precipds.lon - glon) ** 2 +
                        (precipds.lat - glat) ** 2)
            temp_0 = tempds.isel(points=np.argmin(c_tempds.data))
            precip_0 = precipds.isel(points=np.argmin(c_precipds.data))
            temp = temp_0.tas
            temp['lon'] = temp_0.lon
            temp['lat'] = temp_0.lat
            precip = precip_0.pr
            precip['lon'] = precip_0.lon
            precip['lat'] = precip_0.lat

        # Back to [-180, 180] for OGGM
        temp.lon.values = temp.lon if temp.lon <= 180 else temp.lon - 360
        precip.lon.values = precip.lon if precip.lon <= 180 else precip.lon - 360

        # Convert kg m-2 s-1 to mm mth-1 => 1 kg m-2 = 1 mm !!!
        assert 'kg m-2 s-1' in precip.units, 'Precip units not understood'

        ny, r = divmod(len(temp), 12)
        assert r == 0
        dimo = [cfg.DAYS_IN_MONTH[m - 1] for m in temp['time.month']]
        precip = precip * dimo * (60 * 60 * 24)

    process_gcm_data(gdir, filesuffix=filesuffix, prcp=precip, temp=temp,
                     source=filesuffix, **kwargs)


###########
    
# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='ERROR')
rgi_version = '62'

cfg.PARAMS['border'] = 160 # changed for OGGM v16

# I got a strange KeyError: 'dl_verify_data_cluster.klima.uni-bremen.de' (only for RGI reg 03 and  'gfdl-esm4' 'ssp370')
cfg.PARAMS['dl_verify'] = False 
cfg.PARAMS['continue_on_error'] = False

cfg.PARAMS['use_multiprocessing']=True
cfg.PARAMS['store_model_geometry'] = True
           



cfg.PATHS['working_dir'] = utils.gettempdir('OGGM_gcm_run', reset=False)


# Go - get the pre-processed glacier directories
# TODO -> need to change the preprocessed directory to that here when it is available ... 
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.2/elev_bands/W5E5_spinup'
# old v2023.1 version: base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5_spinup'
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=5, prepro_border=160,
                                          prepro_base_url=base_url, prepro_rgi_version=rgi_version)

ALL_DIAGS = ['volume', 'volume_bsl', #'volume_bwl',
             'area', 'length',  'calving', 'calving_rate',
             'off_area', 'on_area', 'melt_off_glacier',
             'melt_on_glacier', 'liq_prcp_off_glacier', 'liq_prcp_on_glacier',
             'snowfall_off_glacier', 'snowfall_on_glacier', 'model_mb',
             'residual_mb', 'snow_bucket']
# Add debug vars
cfg.PARAMS['store_diagnostic_variables'] = ALL_DIAGS

gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=5, prepro_border=160,
                                              prepro_base_url=base_url, prepro_rgi_version=rgi_version)


dir_path = 'https://cluster.klima.uni-bremen.de/~lschuster/provide/gfdl-esm2m_oversh_stab_uni_bern/flat/2023.2'
scenarios = ['stab_T15',
             'oversh_T20OS15',
             'oversh_T25OS15',
             'oversh_T30OS15',
             'stab_T20',
             'stab_T25',
             'stab_T30']

for scenario in scenarios:

    fp = f'{dir_path}/gfdl-esm2m_{scenario}_pr_global_monthly_flat_glaciers.nc'
    ft = f'{dir_path}/gfdl-esm2m_{scenario}_tas_global_monthly_flat_glaciers.nc'
    ft = utils.file_downloader(ft)
    fp = utils.file_downloader(fp)
    rid = f'_gfdl-esm2m_{scenario}_endyr_2500_bc_2000_2019'
    workflow.execute_entity_task(process_cmip_data, gdirs, 
                                 filesuffix=rid,  # recognize the climate file for later
                                 fpath_temp=ft,  # temperature projections
                                 fpath_precip=fp,  # precip projections
                                 year_range=('2000', '2019'),
                                 y0=2000-10,y1=2500) # new this makes the processing a bit faster


2023-05-16 10:37:19: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-05-16 10:37:19: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-05-16 10:37:19: oggm.cfg: Multiprocessing: using all available processors (N=32)
2023-05-16 10:37:19: oggm.cfg: PARAMS['border'] changed from `80` to `160`.
2023-05-16 10:37:19: oggm.cfg: Multiprocessing switched ON after user settings.
2023-05-16 10:37:19: oggm.cfg: PARAMS['store_model_geometry'] changed from `False` to `True`.
2023-05-16 10:37:19: oggm.workflow: init_glacier_directories from prepro level 5 on 8 glaciers.
2023-05-16 10:37:19: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 8 glaciers
2023-05-16 10:37:20: oggm.cfg: PARAMS['store_diagnostic_variables'] changed from `['volume', 'volume_bsl', 'volume_bwl', 'area', 'length', 'calving', 'calving_rate', 'off_area', 'on_area', 'melt_off_glacier', 'melt_on_glacier', 'liq_prcp_off_glacier', 'liq_prcp_on_glacier', 'snow

In [4]:
gdir = gdirs[0]

In [15]:
cfg.PARAMS['continue_on_error'] = True


2023-05-16 10:37:59: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.


In [16]:
for gdir in gdirs: 
    tasks.run_with_hydro(gdir, run_task=tasks.run_from_climate_data,
                     climate_filename='climate_historical',
                     ys=2000, init_model_yr=2000,
                     store_monthly_hydro=True,
                     init_model_filesuffix='_spinup_historical',
                     ref_geometry_filesuffix='_spinup_historical',
                     ref_area_yr = 2000,
                     output_filesuffix='_historical_from_2000_run')

    scenarios = ['stab_T15',
                 'oversh_T20OS15',
                 'oversh_T25OS15',
                 'oversh_T30OS15',
                 'stab_T20',
                 'stab_T25',
                 'stab_T30']
    for scenario in scenarios:
        rid = f'_gfdl-esm2m_{scenario}_endyr_2500_bc_2000_2019'
        tasks.run_with_hydro(gdir,
                 run_task=tasks.run_from_climate_data,
                             ys=2020, # this is important! Start from 2020 glacier
                             ye=2499,
                 # use gcm_data, not climate_historical
                 climate_filename='gcm_data',
                 # use the chosen scenario
                 climate_input_filesuffix=rid,
                 # we start from the previous run, 
                 init_model_filesuffix='_historical_from_2000_run',
                 ref_geometry_filesuffix='_historical_from_2000_run', 
                 ref_area_yr = 2000,
                 # recognize the run for later
                 output_filesuffix=f'_future_run{rid}',
                 # add monthly diagnostics
                 store_monthly_hydro=True);


        utils.merge_consecutive_run_outputs(gdir,
                                input_filesuffix_1='_historical_from_2000_run',
                                input_filesuffix_2=f'_future_run{rid}',
                                output_filesuffix=f'_merged_from_2000_run{rid}',
                                delete_input=False,
                               ) # we will delete that later


2023-05-16 10:37:59: oggm.core.flowline: FileNotFoundError occurred during task run_from_climate_data_historical_from_2000_run on RGI60-10.00002: [Errno 2] No such file or directory: b'/tmp/OGGM/OGGM_gcm_run/per_glacier/RGI60-10/RGI60-10.00/RGI60-10.00002/model_geometry_spinup_historical.nc'
2023-05-16 10:37:59: oggm.core.flowline: InvalidWorkflowError occurred during task run_with_hydro_historical_from_2000_run on RGI60-10.00002: The run task (run_from_climate_data) did not run successfully.
2023-05-16 10:37:59: oggm.core.flowline: FileNotFoundError occurred during task run_from_climate_data_future_run_gfdl-esm2m_stab_T15_endyr_2500_bc_2000_2019 on RGI60-10.00002: [Errno 2] No such file or directory: b'/tmp/OGGM/OGGM_gcm_run/per_glacier/RGI60-10/RGI60-10.00/RGI60-10.00002/model_geometry_historical_from_2000_run.nc'
2023-05-16 10:37:59: oggm.core.flowline: InvalidWorkflowError occurred during task run_with_hydro_future_run_gfdl-esm2m_stab_T15_endyr_2500_bc_2000_2019 on RGI60-10.00002: 